In [2]:
#This script computes the HWMI based on daily fields for a specific period in cross-validation. 
import matplotlib
from scipy import signal
import os
import datetime
import time
import copy
import shutil
import sys
from function_read import *
import numpy as np
from scipy.stats import gaussian_kde
import xarray as xr
import calendar
import locale
from netCDF4 import num2date, date2num
from joblib import Parallel, delayed
import joblib
from netCDF4 import num2date, date2num, Dataset
from numpy import dtype
from HWMI import *

In [3]:
var='tmax'
expname='ERA5'
memb_str='0' # there are not members for ERA5
nrealisation=1
# Region #
reg_name ='Europe'
min_lon = -15
min_lat = 25
max_lon = 60
max_lat = 70
lats_bnds = np.array([min_lat,max_lat])
lons_bnds = np.array([min_lon, max_lon])
# Season #
season='15MJJA'
season_start_day=[5,15]
season_start_day=[8,31]
nday=109 #15 of May to 31st of Aug
# Period #
ref_year1=1981
ref_year2=1983
nyear=len(range(ref_year1,ref_year2))+1


In [4]:
low=False # if data with 1 degree resolution is used (True) or not (False)

if (low==True):
    path="/work/csp/vt17420/ERA5/daily/russo/"+var+"_r360x180/"
    # Where the outputs will be saved
    dirout="/work/csp/vt17420/ERA5/daily/russo/"+var+"_r360x180/HWMI/"
    lons_reg=np.arange(min_lon,max_lon,1)
    lats_reg=np.arange(min_lat,max_lat,1)
    nlon=len(lons_reg)
    nlat=len(lats_reg)
else:
    path="/work/csp/vt17420/ERA5/daily/global/"+var+"/"
    dirout="/data/csp/vt17420/CLINT_PRODUCTS/"+expname+"_HWMI_"+var+"/"
    lons_reg=np.arange(min_lon,max_lon+0.25,0.25)
    lats_reg=np.arange(min_lat,max_lat+0.25,0.25)
    nlon=len(lons_reg)
    nlat=len(lats_reg)

In [5]:
data=np.zeros((nlon,nlat,nyear,nday,1)) # daily data for all the years
for iyear,year in enumerate(range(ref_year1,ref_year2)):
    days_may=np.linspace(15, 31, num=17)
    obs1=xr.open_dataset(path+var+"_"+str(year)+'05.nc')
    if low==False:
        obs1 = obs1.sel(latitude=slice(max_lat,min_lat),longitude=slice(min_lon,max_lon))
    obs=obs1.sel(time=obs1.time.time.dt.day.isin(days_may))
    # selection of the days in june, july and august
    for mon in range(6,9):
        obs1=xr.open_dataset(path+var+"_"+str(year)+str(mon).zfill(2)+'.nc')
        if low==False:
            obs1 = obs1.sel(latitude=slice(max_lat,min_lat),longitude=slice(min_lon,max_lon))
        obs=xr.merge(([obs,obs1]))
    data[:,:,iyear,:,0]=np.transpose(obs.to_array()[0,:,:,:],[2,1,0])

In [6]:
### PERCENTILE THRESHOLD
#percent_thresh = 95
percent_thresh = 90
cv='CV'
cv_str = cv
### MINIMAL DURATION OF A HW
#duration_min = 5
duration_min = 3

In [7]:
nmemb=data.shape[4]
ndayseas = nday//duration_min +1
HWMI = np.zeros((nyear,nmemb,nlat,nlon))
ndayexedthreshold = np.zeros((nyear,nmemb,nlat,nlon))
subHWarray = np.zeros((nyear, ndayseas, nmemb, nlat, nlon))
#HW = np.zeros((nyear,nmemb,nday,nlat,nlon))
#DDthreshold = np.zeros((nyear,nmemb,nlat,nlon))
#fitsubHWarray = np.zeros((nyear, ndayseas, nmemb, nlat, nlon))
#impossible_fit_list = []
#sstMeanarray = np.zeros((nyear,nmemb,nlat,nlon))

def parallelized_HWMIs_computation(ilat, ilon):    #, HWMI, ndayexedthreshold, DDthreshold, fitsubHWarray, subHWarray):
        HWMIyear, HWlstyear, HWstartmembyear, HWendmembyear, ndayexedthresholdyear, DDthresholdyear, subHWarrayyear, fitsubHWarrayyear, sstMeanarrayyear, impossible_fit = calc_HWMIyear(data[ilon,ilat,:,:,:], cross_valid = cv, percent_thresh = percent_thresh, duration_min = duration_min)
        #print HWMIyear
        HWMI[:,:,ilat,ilon]=np.array(HWMIyear)
        ndayexedthreshold[:,:,ilat,ilon]=np.array(ndayexedthresholdyear)
        subHWarray[:,:, :, ilat,ilon]=np.array(subHWarrayyear)
        #DDthreshold[:,:,ilat,ilon]=np.array(DDthresholdyear)
        #fitsubHWarray[:,:, :, ilat,ilon]=np.array(fitsubHWarrayyear)
        #sstMeanarray[:,:,ilat,ilon]=np.array(sstMeanarrayyear)
        #impossible_fit_list.append((impossible_fit,ilat,ilon))
        return()

Parallel(n_jobs=-1, timeout = 5*3600, verbose = 20, require='sharedmem', mmap_mode='w+')(delayed(parallelized_HWMIs_computation)(ilat, ilon) for ilat in range(nlat) for ilon in range(nlon))


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elaps

[Parallel(n_jobs=-1)]: Done 133 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 134 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 135 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 139 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 140 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 142 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 143 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 145 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 147 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 148 tasks      | elapsed:    1.6s
[Paralle

[Parallel(n_jobs=-1)]: Done 288 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 289 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 290 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 291 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 292 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 293 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 294 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 295 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 296 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 297 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 298 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 299 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 300 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 301 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 302 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 303 tasks      | elapsed:    3.1s
[Paralle

[Parallel(n_jobs=-1)]: Done 428 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 429 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 430 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 431 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 432 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 433 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 435 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 436 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 437 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 438 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 439 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 440 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 441 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 443 tasks      | elapsed:    4.3s
[Paralle

[Parallel(n_jobs=-1)]: Done 597 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 598 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 599 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 601 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 602 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 603 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 604 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 605 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 606 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 607 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 608 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 609 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 610 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 611 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 612 tasks      | elapsed:    6.0s
[Paralle

...

[Parallel(n_jobs=-1)]: Done 53939 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 53940 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 53941 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 53942 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 53943 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 53944 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 53945 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 53946 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 53947 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 53948 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 53949 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 53950 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 53951 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 53952 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 53953 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 53954 tasks 

[Parallel(n_jobs=-1)]: Done 54080 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54081 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54082 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54083 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54084 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54085 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54086 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54087 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54088 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54089 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54090 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54091 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54092 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54093 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54094 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54095 tasks 

[Parallel(n_jobs=-1)]: Done 54234 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54235 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54236 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54237 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54238 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54239 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54240 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54241 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54242 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54243 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54244 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54245 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54246 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54247 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54248 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54249 tasks 

[Parallel(n_jobs=-1)]: Done 54365 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54366 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54367 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54368 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54369 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54370 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54371 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54372 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54373 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54374 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54375 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54376 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54377 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54378 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54379 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 54380 tasks 

[(),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),


In [8]:
if os.path.isdir(dirout)==False:
    os.makedirs(dirout)
years=range(ref_year1,ref_year2)
j=0
for i,iyear in enumerate(range(ref_year1, ref_year2+1)):
    #print iyear
    parameters_str = reg_name+"_"+season+"_"+cv_str+'_percent%i'%(percent_thresh)+'_daymin%i'%(duration_min)+"_ref_"+str(ref_year1)+"_"+str(ref_year2)+"_year_"+str(iyear)
    varout1 = "HWMI"+"_"+var+"_"+parameters_str
    vout1="HWMI"+"_"+var
    fileout=dirout+varout1+".nc"#0%i01, monstart)
    print(fileout)
    if len(glob(fileout))==1:
        os.remove(fileout)
    fout=netCDF4.Dataset(fileout, "w")
    #fin=netCDF4.Dataset(targetflst[iyear])
    lat = fout.createDimension('lat', nlat)
    lon = fout.createDimension('lon', nlon) 
    rea = fout.createDimension('realisation', nrealisation) 
    latitudes = fout.createVariable('lat', np.float32, ('lat',)) 
    longitudes = fout.createVariable('lon', np.float32,  ('lon',)) 

    # Time variable
    timedim = fout.createDimension('time', None) 
    times = fout.createVariable('time', np.float64, ('time',)) 
    times.units = "hours since 1970-01-01 00:00:00"
    times.calendar = 'gregorian' 
    times[:]=date2num(datetime(iyear,season_start_day[0],season_start_day[1]), units=times.units,calendar = times.calendar) ####______HERE_____


    latitudes[:] = lats_reg
    lonaux = lons_reg
    longitudes[:] = lonaux
    latitudes.units = 'degree_north'  
    longitudes.units = 'degree_east' 

    # Create the HWMI 4-d variable
    HWMIfile = fout.createVariable(vout1, np.float32, ('time','realisation','lat','lon'))
    fout.description = 'HWMI index (Russo et al. 2014) for ' + season + ' computed in cross validation'
    fout.history = 'computed from python script by C.Prodhomme & S.Lecestre' + time.ctime(time.time())
    fout.source = 'HWMI for ' + expname
    latitudes.units = 'degree_north'  
    longitudes.units = 'degree_east' 
    HWMIfile.units = 'Probability'

    # Create the nb of days 4-d variable
    expercentfile = fout.createVariable("nbdaygtpercentpct_"+var, np.float32, ('time','realisation','lat','lon')) 
    expercentfile.units = 'Number of days'  

    # Write the HWMI variable
    HWMIaux=HWMI[i:i+1,0:1,:,:]
    HWMIfile[0:1,0:1,:,:]=HWMIaux

    # Write the number of days
    exedaux=ndayexedthreshold[i:i+1,j:j+1,:,:]
    expercentfile[0:1,0:1,:,:]=exedaux 

    # Write the DDthreshold
    #DDthresholdfile=DDthreshold
    fout.close()

/data/csp/vt17420/CLINT_PRODUCTS/ERA5_HWMI_tmax/HWMI_tmax_Europe_15MJJA_CV_percent90_daymin3_ref_1981_1983_year_1981.nc
/data/csp/vt17420/CLINT_PRODUCTS/ERA5_HWMI_tmax/HWMI_tmax_Europe_15MJJA_CV_percent90_daymin3_ref_1981_1983_year_1982.nc
/data/csp/vt17420/CLINT_PRODUCTS/ERA5_HWMI_tmax/HWMI_tmax_Europe_15MJJA_CV_percent90_daymin3_ref_1981_1983_year_1983.nc


/tmp/ipykernel_133887/134228956.py:27: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  times[:]=date2num(datetime(iyear,season_start_day[0],season_start_day[1]), units=times.units,calendar = times.calendar) ####______HERE_____
